In [1]:
import numpy

In [2]:
import sys
import subprocess
import ctypes
import time
import numpy

from oamap.schema import *

In [3]:
pydata = [{"particles": [{"x": numpy.random.normal(), "y": numpy.random.normal()} for particle in range(numpy.random.poisson(2))]} for event in range(10000)]
oadata = List(Record({"particles": List(Record({"x": "f8", "y": "f8"}, name="Particle"))}, name="Event")).fromdata(pydata)
arrays = dict(oadata._arrays)
schema = oadata._generator.namedschema()
schema.content["pairs"] = List(
    Tuple([Pointer(schema.content["particles"].content, positions="left"),
           Pointer(schema.content["particles"].content, positions="right")]),
    starts="starts", stops="stops")
    #schema.show()

particlestarts = arrays["object-L-NEvent-Fparticles-B"]
particlestops  = arrays["object-L-NEvent-Fparticles-E"]
particlecounts = particlestops - particlestarts
pairscounts    = (particlecounts)*(particlecounts - 1) / 2   # N*(N-1)/2

pairsoffsets = numpy.empty(len(pairscounts) + 1, int)
pairsoffsets[0] = 0
pairsoffsets[1:] = numpy.cumsum(pairscounts)                 # cumulative sum
arrays["starts"], arrays["stops"] = pairsoffsets[:-1], pairsoffsets[1:]

left  = arrays["left"]  = numpy.empty(pairsoffsets[-1], int)
right = arrays["right"] = numpy.empty(pairsoffsets[-1], int)

In [4]:
'''This is the original triple nested loop'''

for event in range(len(oadata)):
    pairs_i = pairsoffsets[event]
    for particle_i1 in range(particlestarts[event], particlestops[event]):
        for particle_i2 in range(particle_i1 + 1, particlestops[event]):
            left[pairs_i]  = particle_i1
            right[pairs_i] = particle_i2
            pairs_i += 1


In [5]:
'''This is with one loop and numpy'''

start_mask = particlestarts[particlecounts>1]
count_offset_mask = particlecounts[particlecounts>1]
left_n = numpy.empty(1)
right_n = numpy.empty(1)
#How to reduce this loop??
for event in numpy.arange(len(count_offset_mask)): 
    left1, right1 = numpy.triu_indices(count_offset_mask[event], 1)+start_mask[event]
    #print(left1)
    left_n = numpy.hstack((left_n, left1))
    right_n = numpy.hstack((right_n, right1))

left_n = left_n[1:]
right_n = right_n[1:]

In [6]:
'''Proof that it works'''
print(sum(left_n-left))

0.0
